In [1]:
%matplotlib notebook
import matplotlib
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import cm

from matplotlib import colors

from fridge import Fridge
from tes import TES
from QET import QET
from detector import Detector
from electronics import Electronics
from absorber import Absorber
from simulated_noise import simulate_noise
from scipy.optimize import minimize
import numpy as np
import matplotlib.pyplot as plt
from MaterialProperties import TESMaterial

In [2]:
printing = False
fSnolab = Fridge("SNOLAB", 20e-3, 145e-3, 900e-3, 4.8, 0)
# Absorber: Silicon. Height 1mm. Radius 38.1mm. W safety 3mm. 
# Q: W safety removes a huge % of patterned surface area
absorber = Absorber("Si", "square", 1e-3, 10e-3, 3e-3, printing) # same as matlab
eSnolab = Electronics(fSnolab, fSnolab.get_TCP(), fSnolab.get_TMC())
eSLAC = Electronics(fSnolab, fSnolab.get_TMC(), fSnolab.get_TMC(), 5e-3, 6e-3, 25e-9, 25e-9, 4e-12)

tungsten = TESMaterial()

In [3]:
tes_w = 2.5e-6

h_fin = 900e-9 
ahole = 49e-12
sigma = tungsten._gPep_v

T_eq = -100

l_overlaps = []
l_fins = []
tes_lengths = []

l_over_min = 4e-6
l_over_delta = 1e-6

l_fin_min = 25e-6
l_fin_delta = 5e-6

tes_min = 25e-6
tes_delta = 5e-6

while l_over_min < 35e-6:
    l_overlaps.append(l_over_min)
    l_over_min = l_over_min + l_over_delta
    
while l_fin_min < 300e-6:
    l_fins.append(l_fin_min)
    l_fin_min = l_fin_min + l_fin_delta 
    
while tes_min < 200e-6:
    tes_lengths.append(tes_min)
    tes_min = tes_min + tes_delta 

l_fins_mu = []
l_overlaps_mu = []
tes_lengths_mu = []

for l_o in l_overlaps:
    l_overlaps_mu.append(l_o*(10**6))
for l_f in l_fins:
    l_fins_mu.append(l_f*(10**6))
for t in tes_lengths:
    tes_lengths_mu.append(t*(10**6))
    
n_fins = [2, 4]

coverages = [0.001, 0.0025, 0.005, 0.01, 0.02, 0.03, 0.04]

In [4]:
tes_l = 3e-05
l_f =   5e-05
l_over = 8e-06
n_fin =  4
res_n = 0.9599999999999997


tes = TES(tes_l, tes_w, l_over, n_fin, sigma, T_eq, res_n, 0.45, tungsten, printing)
qet = QET( l_f, h_fin, tes, ahole)
                
det = Detector("det name", fSnolab, eSLAC, absorber, qet, tes, 1, 0)
e_res = simulate_noise(det) 

SQUID NOISE:  6.932073938563034e-38
Rl Noise: 2.8014751525796493e-40
Rt Noise: 4.583818889015447e-42
Gtb Noise: 5.479969064872381e-38


In [7]:
tes_l =  2.5e-05
l_f = 5e-05
l_over =  1.4e-05
n_fin = 2
res_n =  0.857142857142857

tes = TES(tes_l, tes_w, l_over, n_fin, sigma, T_eq, res_n, 0.45, tungsten, printing)
qet = QET( l_f, h_fin, tes, ahole)
                
det = Detector("det name", fSnolab, eSLAC, absorber, qet, tes, 1, 0)
e_res = simulate_noise(det) 


SQUID NOISE:  5.969926092374453e-38
Rl Noise: 3.044932241429766e-40
Rt Noise: 4.4483636693494596e-42
Gtb Noise: 5.31803194837203e-38


In [ ]:
for cov in coverages:
    print(cov*100, " % Coverage")
    for n_fin in n_fins:
        print("   -- ", n_fin, " Fins")
        min_resolution = 200

        for l_f in l_fins:
            for tes_l in tes_lengths:
                for l_over in l_overlaps:
                    perim = tes_l*2 + 14e-6*2 - 6e-6*n_fin
                    overlap_p = n_fin*l_over*2
                    if overlap_p > perim: 
                        continue
            
                    # Calculate QET Active Area
                    wempty = 6e-6
                    wempty_tes = 7.5e-6
                    nhole = 3*n_fin 
                    afin_empty = n_fin * l_f * wempty + 2 * tes_l * wempty_tes + nhole * ahole                          
                    a_fin = np.pi*l_f*(l_f + (tes_l/2)) - afin_empty   
            
                    N_tes = cov*absorber._SA/a_fin
                    N_tes = int(N_tes)
         
            
                    res_n = tungsten._rho_electrical*tes_l/(tes_w*(40e-9)*N_tes)
            
                    tes = TES(tes_l, tes_w, l_over, n_fin, sigma, T_eq, res_n, 0.45, tungsten, printing)
                    qet = QET( l_f, h_fin, tes, ahole)
                
                    det = Detector("det name", fSnolab, eSLAC, absorber, qet, tes, 1, 0)
                    e_res = simulate_noise(det)
            
                    if e_res < min_resolution: 
                        min_resolution = e_res
                        tes_opt = tes
                        qet_opt = qet
                        det_opt = det 
            
        print("BASELINE RESOLUTION: ", min_resolution) 

        pActiveArea = det_opt._SA_active/det_opt._absorber.get_SA()
        pPassiveArea = det_opt._SA_passive/det_opt._absorber.get_SA()

        res_4fin.append(min_resolution)
        coverages_4fin.append(pActiveArea)
        rn_4fin.append(tes_opt._total_res_n)
        tes_l_4fin.append(tes_opt._l)
        tot_eff_4fin.append(det_opt._eEabsb)
        pass_area_4fin.append(pPassiveArea)
        tot_cov_4fin.append(det_opt._fSA_qpabsorb)
        qp_eff_4fin.append(det_opt._qet._eQPabsb)

        print("TES Length             ", tes_opt._l)
        print("Fin Length             ", qet_opt._l_fin)
        print("Overlap                ", qet_opt.l_overlap)
        print("N Fins                 ", tes_opt._n_fin)
        print("Percent Active SA      ", pActiveArea)
        print("Percent Passive SA     ", pPassiveArea)
        print("Percent QP Absorb SA   ", det_opt._fSA_qpabsorb)
        print("N tes                  ", det_opt._tes._nTES)
        print("Rn                     ", det_opt._tes._total_res_n )
        print("fQP Absorb             ", det_opt._qet._eQPabsb)
        print("Phonon Absorption Time ", det_opt._t_pabsb)
        print("Time ETF               ", det_opt._tes._tau_etf)
        print("Cells Fit              ", det_opt._cells_fit)

In [ ]:
tes_l = 3e-05
l_f =   5e-05
l_over = 8e-06
n_fin =  4
res_n = 0.9599999999999997


tes = TES(tes_l, tes_w, l_over, n_fin, sigma, T_eq, res_n, 0.45, tungsten, printing)
qet = QET( l_f, h_fin, tes, ahole)
                
det = Detector("det name", fSnolab, eSLAC, absorber, qet, tes, 1, 0)
e_res = simulate_noise(det)